<a href="https://colab.research.google.com/github/d7lewandowski/pyspark-script/blob/main/dataalgoritmswithpyspark/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=74e9bd48fbc49020a909b01100282656b7b4c2362a51e4db1863eb42d2e6541b
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local').appName('ETL').getOrCreate()

# EXTRACTION (source data)

In [ ]:
input_path = '/content/drive/MyDrive/Colab Pyspark/data/census_2010.json'
census_df = spark.read.json(input_path)
census_df.count()

101

In [ ]:
census_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- females: long (nullable = true)
 |-- males: long (nullable = true)
 |-- year: long (nullable = true)



In [ ]:
census_df.show(200)

+---+-------+-------+----+
|age|females|  males|year|
+---+-------+-------+----+
|  0|1994141|2085528|2010|
|  1|1997991|2087350|2010|
|  2|2000746|2088549|2010|
|  3|2002756|2089465|2010|
|  4|2004366|2090436|2010|
|  5|2005925|2091803|2010|
|  6|2007781|2093905|2010|
|  7|2010281|2097080|2010|
|  8|2013771|2101670|2010|
|  9|2018603|2108014|2010|
| 10|2023289|2114217|2010|
| 11|2026352|2118390|2010|
| 12|2037286|2132030|2010|
| 13|2060100|2159943|2010|
| 14|2089651|2195773|2010|
| 15|2117689|2229339|2010|
| 16|2146942|2263862|2010|
| 17|2165852|2285295|2010|
| 18|2168175|2285990|2010|
| 19|2159571|2272689|2010|
| 20|2151448|2259690|2010|
| 21|2140926|2244039|2010|
| 22|2133510|2229168|2010|
| 23|2132897|2218195|2010|
| 24|2135789|2208905|2010|
| 25|2136497|2197148|2010|
| 26|2137711|2185169|2010|
| 27|2131362|2167654|2010|
| 28|2113361|2142006|2010|
| 29|2088593|2111921|2010|
| 30|2065883|2083642|2010|
| 31|2043293|2055863|2010|
| 32|2027525|2034632|2010|
| 33|2022761|2023579|2010|
|

# Transformation

In [ ]:
seniors = census_df[census_df['age'] > 54]
seniors.count()

46

In [ ]:
seniors.show(5)

+---+-------+-------+----+
|age|females|  males|year|
+---+-------+-------+----+
| 55|2167706|2059204|2010|
| 56|2106460|1989505|2010|
| 57|2048896|1924113|2010|
| 58|2001049|1869486|2010|
| 59|1957350|1819943|2010|
+---+-------+-------+----+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import lit

seniors_final = seniors.withColumn('total', lit(seniors.males + seniors.females))
seniors_final.show(10)

+---+-------+-------+----+-------+
|age|females|  males|year|  total|
+---+-------+-------+----+-------+
| 55|2167706|2059204|2010|4226910|
| 56|2106460|1989505|2010|4095965|
| 57|2048896|1924113|2010|3973009|
| 58|2001049|1869486|2010|3870535|
| 59|1957350|1819943|2010|3777293|
| 60|1908602|1765522|2010|3674124|
| 61|1859542|1710843|2010|3570385|
| 62|1794846|1642438|2010|3437284|
| 63|1706899|1553454|2010|3260353|
| 64|1604801|1452106|2010|3056907|
+---+-------+-------+----+-------+
only showing top 10 rows



# Load data to target

In [ ]:
seniors_final\
 .write\
 .format("jdbc")\
 .option("driver", "com.mysql.jdbc.Driver")\
 .mode("overwrite")\
 .option("url", "jdbc:mysql://localhost/testdb")\
 .option("dbtable", "seniors")\
 .option("user", "root")\
 .option("password", "root_password")\
 .save()